In [1]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-cased')

import numpy as np
import torch
from datasets import load_dataset
from datasets.arrow_dataset import concatenate_datasets

data_set = load_dataset("nlu_evaluation_data",split='train')

data_alarm_audio= data_set.filter(lambda example: example['scenario'] == 'alarm' or example['scenario'] == 'audio')

from transformers import BertModel, BertTokenizer
model_name = 'bert-base-uncased'
labels = ["alarm_query","alarm_remove","alarm_set","audio_volume_down","audio_volume_mute","audio_volume_other","audio_volume_up"]

def createEmbeddings(input):
  tokenizer = BertTokenizer.from_pretrained(model_name)
  # load
  embedding_rep = {}
  model = BertModel.from_pretrained(model_name)
  for i in input :
      input_text = i
      input_ids = tokenizer.encode(input_text, add_special_tokens=True)
      input_ids = torch.tensor([input_ids])
      embedding_rep[i] = model(input_ids).last_hidden_state
  return embedding_rep

labels = ["alarm_query","alarm_remove","alarm_set","audio_volume_down","audio_volume_mute","audio_volume_other","audio_volume_up"]

label_embeddings = createEmbeddings(labels)

label_embeddings["alarm_remove"].shape

input_text=[]
for entry in data_alarm_audio:
  input_text.append(entry["text"])

input_text[0]

sentence_embeddings = createEmbeddings(input_text)

sentence_embeddings["wake me up at five am this week"].shape

def doCross(label_embedd,sentence_embedd):
  return np.dot(label_embedd,sentence_embedd)

ex1 = sentence_embeddings["wake me up at five am this week"][:, 0:1,:].detach().numpy()
ex2_1 = label_embeddings["alarm_query"][:, 0:1,:].detach().numpy()
ex2_2 = label_embeddings["alarm_set"][:, 0:1,:].detach().numpy()

print(doCross(np.squeeze(np.asarray(ex1)),np.squeeze(np.asarray(ex2_1))))

print(doCross(np.squeeze(np.asarray(ex1)),np.squeeze(np.asarray(ex2_2))))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default
Reusing dataset nlu_evaluation_data (/home/sid/.cache/huggingface/datasets/nlu_evaluation_data/default/1.1.0/0416a5876d8240bd571f2bc2ad421cf6e6e88d938f8dcb5fd87b5af6033d6282)


  0%|          | 0/26 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.pred

173.63988
177.25735
